# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

## YelpReviewFull 数据集

**Hugging Face 数据集：[ YelpReviewFull ](https://huggingface.co/datasets/yelp_review_full)**

### 数据集摘要

Yelp评论数据集包括来自Yelp的评论。它是从Yelp Dataset Challenge 2015数据中提取的。

### 支持的任务和排行榜
文本分类、情感分类：该数据集主要用于文本分类：给定文本，预测情感。

### 语言
这些评论主要以英语编写。

### 数据集结构

#### 数据实例
一个典型的数据点包括文本和相应的标签。

来自YelpReviewFull测试集的示例如下：

```json
{
    'label': 0,
    'text': 'I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!'
}
```

#### 数据字段

- 'text': 评论文本使用双引号（"）转义，任何内部双引号都通过2个双引号（""）转义。换行符使用反斜杠后跟一个 "n" 字符转义，即 "\n"。
- 'label': 对应于评论的分数（介于1和5之间）。

#### 数据拆分

Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。

## 下载数据集

In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

d:\anaconda3\envs\llm-test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 50000/50000 [00:00<00:00, 950589.26 examples/s]


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [3]:
dataset["train"][10]

{'label': 0,
 'text': "Owning a driving range inside the city limits is like a license to print money.  I don't think I ask much out of a driving range.  Decent mats, clean balls and accessible hours.  Hell you need even less people now with the advent of the machine that doles out the balls.  This place has none of them.  It is april and there are no grass tees yet.  BTW they opened for the season this week although it has been golfing weather for a month.  The mats look like the carpet at my 107 year old aunt Irene's house.  Worn and thread bare.  Let's talk about the hours.  This place is equipped with lights yet they only sell buckets of balls until 730.  It is still light out.  Finally lets you have the pit to hit into.  When I arrived I wasn't sure if this was a driving range or an excavation site for a mastodon or a strip mining operation.  There is no grass on the range. Just mud.  Makes it a good tool to figure out how far you actually are hitting the ball.  Oh, they are cash 

In [4]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [5]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

,label,text
0,3 stars,"I'm giving a 3 star rating because it isn't good enough to deserve 4 and not so bad that I'd give it a two. \n\nMy wife and I will stop in on a Thursday night to get the \""Date Night\"" special. An appetizer, two small salads and a pizza for $20. Great deal for a thrifty couple. Any other night would be too much of a strain on the pocketbook as items tend to be a little pricey. \n\nYeah the wait can be long, service is slow and they pack too many people in a small space. The outdoor patio is nice during the right time of year and there has been decent live entertainment at the bar. Wait staff can be a bit odd but not enough to bother me. \n\nOne night they ran out of mozzarella for the appetizer. I know, a pizza place ran out of mozzarella right? They didn't tell us until way too late but made up for it with a free dessert. \n\nBottom line, I will go back for the special but not much else."
1,5 stars,"Had a great experience here AGAIN! Wonderful peanut butter bacon burger, would definitely order it again! Best burgers so far BBQ bacon and peanut butter bacon. I have tried the bleu cheese but it's a little strong for me! Definitely a must try!!"
2,5 stars,"Good food, amazing service. We even got to tour the kitchen."
3,1 star,"What a brutal dining experience. In my many, many years of dining out, I have never experienced such terrible, unfriendly and unwelcoming service. We walked in to the restaurant and it had a distinct odor, much like that of natural gas. We hoped for the best and definitely got the worst. From the moment we sat down, my friend and I were assaulted by murderous glances from the owner. Was it because we brought our children to dine in his establishment? How dare we? We had to ask for bread, drinks, napkins a couple of times before receiving them. The waitress was overwhelmed and not too kind either. The food came out in waves, not all together as you would expect when dining in a restaurant. The adult's food arrived at least 8 minutes before the children's, you'd think it would be the opposite. The food was actually quite good, yet not good enough, to overcome the terrible discomfort that we felt being in this restaurant. Truly, I would have felt more welcome in McDonald's. \nIn this economy, you should welcome each and every paying customer into your restaurant with open arms, or at least a smile. A little kindness goes a long way....."
4,3 stars,"The positive: love the Oreo shake its thick, creamy and rich definitely will order it again when in town. The frozen chocolate kids devour it in nano seconds. They loved it. Kids love the burgers juicy and moist. Th crab cakes sandwiches not bad but I had better. \n\nThe Negative: kids meal pasta with sausage not good at all. My three girls didn't touch it after a few bites. It had no flavor so bland and nothing that resembles pasta sauce at all yuk! The Caesar salad with steak, steak is bland and chewy, could use a little bit more Caesar dressing . \nOverall I will go there again just for the Oreo shakes and maybe try their other desserts. I will skip their food its $$$$ not much flavor food just isn't good."
5,2 star,"We walked in around four thirty pm. Maybe it was just a bad evening for them, but no-one greeted us at the door. When a waitress did show up, she just waved her hand and said, \"" you can sit anywhere\"" So we seated ourselves\n\nWe were taken care of promptly, no problems there. However when our sushi arrived, we had to ask for soy sauce, wasabi and ginger to be served with it. The waitress seemed surprised that we wanted it\n\nOverall, the sushi was average. The service was less than stellar. The servers really need to step it up. I won't be back."
6,3 stars,"We went again last night...we sat in the back row. The seats are really, really, uncomfortable. They seem to have updated the hosting seating process...so we arrived 6:45 for a 7:50 show to be sure we had good seats...luckily I am used to fly SWA and I was able to bar

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<?, ?B/s]
d:\anaconda3\envs\llm-test\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\17972\.cache\huggingface\hub\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config.json: 100%|██████████| 570/570 [00:00<?, ?B/s] 
vo

In [10]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,5 stars,"Wonderful fresh Persian food! But even better, great customer service!! We arrived after the kitchen closed, but to help us eat a healthy, hearty meal, Aram took our order and gave us great take out. Generous portions and kindness to strangers are much appreciated. Don't miss Royal Persis and try the Fesenjoon.","[101, 20361, 4489, 3886, 2094, 106, 1252, 1256, 1618, 117, 1632, 8132, 1555, 106, 106, 1284, 2474, 1170, 1103, 3119, 1804, 117, 1133, 1106, 1494, 1366, 3940, 170, 8071, 117, 1762, 1183, 7696, 117, 25692, 1306, 1261, 1412, 1546, 1105, 1522, 1366, 1632, 1321, 1149, 119, 9066, 13149, 8924, 1105, 18569, 1106, 15712, 1132, 1277, 12503, 119, 1790, 112, 189, 5529, 1787, 14286, 4863, 1105, 2222, 1103, 11907, 3792, 5077, 1320, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


### 数据抽样

使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [23]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(650000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(50000))

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [24]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [26]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=5000)

In [27]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [28]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [29]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [30]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch", 
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=5000)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

## 使用 nvidia-smi 查看 GPU 使用

为了实时查看GPU使用情况，可以使用 `watch` 指令实现轮询：`watch -n 1 nvidia-smi`:

```shell
Every 1.0s: nvidia-smi                                                   Wed Dec 20 14:37:41 2023

Wed Dec 20 14:37:41 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   64C    P0              69W /  70W |   6665MiB / 15360MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A     18395      C   /root/miniconda3/bin/python                6660MiB |
+---------------------------------------------------------------------------------------+
```

In [32]:
trainer.train()

  0%|          | 167/243750 [3:49:18<5574:26:59, 82.39s/it]
d:\anaconda3\envs\llm-test\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
                                                        
  2%|▏         | 5000/243750 [09:42<7:37:36,  8.70it/s]    

{'loss': 1.0327, 'learning_rate': 4.8974358974358975e-05, 'epoch': 0.06}


                                                        
  4%|▍         | 10000/243750 [19:26<7:31:16,  8.63it/s]   

{'loss': 0.932, 'learning_rate': 4.7948717948717955e-05, 'epoch': 0.12}


                                                         
  6%|▌         | 15000/243750 [29:08<7:21:44,  8.63it/s]   

...

                                                         
 98%|█████████▊| 240000/243750 [7:52:53<07:07,  8.77it/s]   

{'loss': 0.7478, 'learning_rate': 7.692307692307694e-07, 'epoch': 2.95}


100%|██████████| 243750/243750 [8:00:13<00:00,  8.67it/s]
















































































































































































































































































































































































































































































































































































































































































































































































































































































































































































{'eval_loss': 0.8057957291603088, 'eval_accuracy': 0.66116, 'eval_runtime': 259.6802, 'eval_samples_per_second': 192.545, 'eval_steps_per_second': 24.068, 'epoch': 3.0}
{'train_runtime': 29072.7788, 'train_samples_per_second': 67.073, 'train_steps_per_second': 8.384, 'train_loss': 0.832849788661859, 'epoch': 3.0}


TrainOutput(global_step=243750, training_loss=0.832849788661859, metrics={'train_runtime': 29072.7788, 'train_samples_per_second': 67.073, 'train_steps_per_second': 8.384, 'train_loss': 0.832849788661859, 'epoch': 3.0})

In [33]:
test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(10000))

In [34]:
trainer.evaluate(test_dataset)

100%|██████████| 1250/1250 [00:51<00:00, 24.41it/s]


{'eval_loss': 0.8071225881576538,
 'eval_accuracy': 0.6661,
 'eval_runtime': 51.482,
 'eval_samples_per_second': 194.243,
 'eval_steps_per_second': 24.28,
 'epoch': 3.0}

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [35]:
trainer.save_model(f"{model_dir}/finetuned-trainer")

In [36]:
trainer.save_state()

## Homework: 使用完整的 YelpReviewFull 数据集训练，对比看 Acc 最高能到多少

Acc提升到了0.6661